In [4]:
import numpy
import scipy.special
import scipy.ndimage
import matplotlib.pyplot

#神经网络类
class nerualNetwork:
    
    #初始化神经网络
    def __init__(self, inputnodes, hiddennodes, outputnodes, learningrate):
        #为输入层，隐藏层，输出层设定大小
        self.inodes    = inputnodes
        self.hnodes    = hiddennodes
        self.onodes    = outputnodes
        
        #设定学习率
        self.lr        = learningrate
        
        #生成链接矩阵
        #根据正态分布采样权重:        采样中心，采样范围，              ，矩阵大小
        self.wih = numpy.random.normal( 0.0, pow(self.hnodes, -0.5), (self.hnodes, self.inodes) )
        self.who = numpy.random.normal( 0.0, pow(self.onodes, -0.5), (self.onodes, self.hnodes) )
        
        #sigmoid func
        self.sigmoid   = lambda x: scipy.special.expit(x)
        pass
    
    #训练
    def train(self, inputs_list, targets_list):
        #Part 1.*:查询输出
        inputs         = numpy.array(inputs_list, ndmin=2).T
        targets        = numpy.array(targets_list, ndmin=2).T
        ########->hidden layer
        hidden_outputs = self.sigmoid(numpy.dot(self.wih, inputs))
        ########->output layer
        final_outputs  = self.sigmoid(numpy.dot(self.who, hidden_outputs))
        
        #Part 2.0:计算误差
        #output <- result
        output_errors  = targets - final_outputs
        #hidden <- output
        hidden_errors  = numpy.dot(self.who.T, output_errors)
        
        #Part 2.1:更新权重
        ########[hidden, output]
        self.who += self.lr * numpy.dot((output_errors * final_outputs * (1.0 - final_outputs)), numpy.transpose(hidden_outputs))
        ########[input , hidden]
        self.wih += self.lr * numpy.dot((hidden_errors * hidden_outputs * (1.0 - hidden_outputs)), numpy.transpose(inputs))
        pass
    
    #查询
    def query(self, inputs_list):
        #input_list -> 2d array
        inputs         = numpy.array(inputs_list, ndmin=2).T
        
        #-> hidden layer
        hidden_inputs  = numpy.dot(self.wih, inputs)
        hidden_outputs = self.sigmoid(hidden_inputs)
        
        #-> output layer
        final_inputs   = numpy.dot(self.who, hidden_outputs)
        final_outputs  = self.sigmoid(final_inputs)
        
        return final_outputs

In [3]:
numpy.random.rand(3, 3) - 0.5

array([[-0.24665725,  0.0517408 , -0.47720078],
       [ 0.11514794, -0.13782948, -0.33221127],
       [-0.22225433, -0.46714535, -0.13511265]])

In [ ]:
data_file = open("mnist_dataset/mnist_train_100.csv", 'r')
data_list = data_file.readlines()
data_file.close()

In [ ]:
len(data_list)

In [ ]:
data_list[0]

In [ ]:
import numpy
import matplotlib.pyplot

all_values = data_list[0].split(',')
image_array = numpy.asfarray(all_values[1:]).reshape((28,28))
matplotlib.pyplot.imshow( image_array, cmap = 'Greys', interpolation = 'None')

In [ ]:
#scale input to range 0.01 to 1.0
values = (( numpy.asfarray(all_values[1:]) ) / 255.0 * 0.99)+ 0.01
print(values)

In [18]:
#训练
inputnodes   = 784
hiddennodes  = 200
outputnodes  = 10

learningrate = 0.1

n = nerualNetwork(inputnodes, hiddennodes, outputnodes, learningrate)
epochs = 5

data_file = open("mnist_test.csv", 'r')
data_list = data_file.readlines()
data_file.close()

#train
for e in range(epochs):
    for record in data_list:
        #以','分割
        all_values = record.split(',')
        #绘图
        #image_array = numpy.asfarray(all_values[1:]).reshape((28,28))
        #matplotlib.pyplot.imshow( image_array, cmap = 'Greys', interpolation = 'None')
        #输入处理
        values = (( numpy.asfarray(all_values[1:]) )/ 255 * 0.99) + 0.01
        #期望值处理
        targets = numpy.zeros(outputnodes) + 0.01
        targets[int(all_values[0])] = 0.99

        n.train(values, targets)
    
print('DONE')

DONE


In [11]:
#旋转
#训练
inputnodes   = 784
hiddennodes  = 200
outputnodes  = 10

learningrate = 0.1

n = nerualNetwork(inputnodes, hiddennodes, outputnodes, learningrate)
epochs = 10

data_file = open("mnist_test.csv", 'r')
data_list = data_file.readlines()
data_file.close()

#train
for e in range(epochs):
    print(e)
    for record in data_list:
        #以','分割
        all_values = record.split(',')
        #绘图
        #image_array = numpy.asfarray(all_values[1:]).reshape((28,28))
        #matplotlib.pyplot.imshow( image_array, cmap = 'Greys', interpolation = 'None')
        #输入处理
        values = (( numpy.asfarray(all_values[1:]) )/ 255 * 0.99) + 0.01
        #旋转
        values_plus10  = scipy.ndimage.interpolation.rotate(values.reshape(28, 28),  10, cval = 0.01, reshape = False)
        values_minus10 = scipy.ndimage.interpolation.rotate(values.reshape(28, 28), -10, cval = 0.01, reshape = False)
        list_plus10  = values_plus10.reshape(1,784).tolist()
        list_minus10 = values_minus10.reshape(1,784).tolist()
        #期望值处理
        targets = numpy.zeros(outputnodes) + 0.01
        targets[int(all_values[0])] = 0.99
        
        n.train(values, targets)
        n.train(list_plus10, targets)
        n.train(list_minus10, targets)
        
print('DONE')

0
1
2
3
4
5
6
7
8
9
DONE


In [12]:
#测试

test_data_file = open("mnist_test.csv", 'r')
test_data_list = test_data_file.readlines()
test_data_file.close()

score = []
for record in test_data_list:
    all_values = record.split(',')
    values = ( numpy.asfarray(all_values[1:]) )/ 255 * 0.99 + 0.01
    correct_label = int(all_values[0])
    #print("Correct label: ", correct_label)
    
    answer = n.query(values)
    label = numpy.argmax(answer)
    #print("Network guess: ", label)
    
    if (label == correct_label):
        score.append(1)
    else:
        score.append(0)

print('DONE')

DONE


In [13]:
scorecard = numpy.asarray(score)
print("Performance = ", scorecard.sum() / scorecard.size)

Performance =  0.9916
